# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("../output_data/city_data.csv")
city_data_df.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Cururupu,-1.83,-44.87,72.12,95,47,0.64,BR,1595141515
1,1,Bathsheba,13.22,-59.52,80.60,78,40,5.70,BB,1595140961
2,2,Tuktoyaktuk,69.45,-133.04,38.57,89,100,3.80,CA,1595140951
3,3,Klaksvík,62.23,-6.59,48.20,87,90,6.20,FO,1595140950
4,4,Hūn,29.13,15.95,80.74,43,0,1.87,LY,1595141020


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
city_locations = city_data_df[["Lat", "Lng"]].astype(float)
city_humidity = city_data_df["Humidity"].astype(float)

In [4]:
# Create a poverty Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(city_locations,weights=city_humidity, 
                                 dissipating=False, max_intensity=500,
                                 point_radius = 10)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Humidity between 50% and 80%.
ideal_city_df = city_data_df.loc[(city_data_df["Max Temp"]>70) & 
                                 (city_data_df["Max Temp"]<80) &
                                 (city_data_df["Wind Speed"]<10) &
                                 (city_data_df["Cloudiness"]==0) &
                                 (city_data_df["Humidity"]>40) & 
                                 (city_data_df["Humidity"]<70)]
#Drop any rows that don't contain all three conditions.
ideal_city_df = ideal_city_df.dropna()
ideal_city_df

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,Mossamedes,-15.20,12.15,70.77,52,0,2.12,AO,1595140954
153,153,Mareeba,-17.00,145.43,77.00,65,0,2.24,AU,1595141433
161,161,Coswig,51.13,13.58,71.60,64,0,1.00,DE,1595141532
186,186,Srednekolymsk,67.45,153.68,74.71,41,0,4.49,RU,1595140937
187,187,Bilibino,68.05,166.44,70.03,43,0,5.28,RU,1595141001
222,222,Corinth,37.94,22.96,79.00,53,0,1.79,GR,1595141539
282,282,Portland,45.52,-122.68,72.00,63,0,1.62,US,1595140757
310,310,Develi,38.39,35.49,78.80,47,0,3.60,TR,1595141551
394,394,Ballina,-28.87,153.57,71.60,68,0,4.10,AU,1595141560
431,431,Vezirköprü,41.14,35.45,73.40,60,0,1.00,TR,1595141566


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_city_df
hotel_df["Hotel Name"] = ""
hotel_df

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
27,27,Mossamedes,-15.20,12.15,70.77,52,0,2.12,AO,1595140954,
153,153,Mareeba,-17.00,145.43,77.00,65,0,2.24,AU,1595141433,
161,161,Coswig,51.13,13.58,71.60,64,0,1.00,DE,1595141532,
186,186,Srednekolymsk,67.45,153.68,74.71,41,0,4.49,RU,1595140937,
187,187,Bilibino,68.05,166.44,70.03,43,0,5.28,RU,1595141001,
222,222,Corinth,37.94,22.96,79.00,53,0,1.79,GR,1595141539,
282,282,Portland,45.52,-122.68,72.00,63,0,1.62,US,1595140757,
310,310,Develi,38.39,35.49,78.80,47,0,3.60,TR,1595141551,
394,394,Ballina,-28.87,153.57,71.60,68,0,4.10,AU,1595141560,
431,431,Vezirköprü,41.14,35.45,73.40,60,0,1.00,TR,1595141566,


In [8]:
params = {
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # make request using url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        pass
hotel_df

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
27,27,Mossamedes,-15.20,12.15,70.77,52,0,2.12,AO,1595140954,iu Hotel Namibe
153,153,Mareeba,-17.00,145.43,77.00,65,0,2.24,AU,1595141433,Jackaroo Motel
161,161,Coswig,51.13,13.58,71.60,64,0,1.00,DE,1595141532,Ferienhof Dittrichs Erben
186,186,Srednekolymsk,67.45,153.68,74.71,41,0,4.49,RU,1595140937,
187,187,Bilibino,68.05,166.44,70.03,43,0,5.28,RU,1595141001,Profilaktoriy Baes
222,222,Corinth,37.94,22.96,79.00,53,0,1.79,GR,1595141539,Hotel King Saron Ξενοδοχείο
282,282,Portland,45.52,-122.68,72.00,63,0,1.62,US,1595140757,Hotel deLuxe
310,310,Develi,38.39,35.49,78.80,47,0,3.60,TR,1595141551,Grand Erciyes Otel
394,394,Ballina,-28.87,153.57,71.60,68,0,4.10,AU,1595141560,Ramada Hotel & Suites by Wyndham Ballina Byron
431,431,Vezirköprü,41.14,35.45,73.40,60,0,1.00,TR,1595141566,


In [9]:
# Drop the rows containing empty Hotel Name
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
hotel_df

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
27,27,Mossamedes,-15.20,12.15,70.77,52,0,2.12,AO,1595140954,iu Hotel Namibe
153,153,Mareeba,-17.00,145.43,77.00,65,0,2.24,AU,1595141433,Jackaroo Motel
161,161,Coswig,51.13,13.58,71.60,64,0,1.00,DE,1595141532,Ferienhof Dittrichs Erben
187,187,Bilibino,68.05,166.44,70.03,43,0,5.28,RU,1595141001,Profilaktoriy Baes
222,222,Corinth,37.94,22.96,79.00,53,0,1.79,GR,1595141539,Hotel King Saron Ξενοδοχείο
282,282,Portland,45.52,-122.68,72.00,63,0,1.62,US,1595140757,Hotel deLuxe
310,310,Develi,38.39,35.49,78.80,47,0,3.60,TR,1595141551,Grand Erciyes Otel
394,394,Ballina,-28.87,153.57,71.60,68,0,4.10,AU,1595141560,Ramada Hotel & Suites by Wyndham Ballina Byron


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(city_locations, weights=city_humidity, 
                                 dissipating=False, max_intensity=500,
                                 point_radius = 10)
fig.add_layer(heat_layer)

markers = gmaps.marker_layer(locations,
    info_box_content= hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))